# Automatic Differentiation in Practice 

------
# Autograd
**Advantages:** Extremely simple, full-featured. 

**Disadvantages:** No GPU support, slower than PyTorch, some quirks we will look at shortly. 

## Basics

In [2]:
import autograd 
import autograd.numpy as np # instead of import numpy as np 

def f1(x): 
    return np.log(x[0]) + x[0]*x[1] - np.sin(x[1])

grad_f = autograd.grad(f1,0) 
x = np.array([1.,2.])
grad_f(x)

array([3.        , 1.41614684])

Let's use finite differences to make sure it worked!

In [3]:
def central_difference(x,func, h = 1e-5): 
    n = x.shape[0]
    gr = np.zeros(n)
        
    for i in range(n):
        delta = np.zeros(n)
        delta[i] += h
        xb, xf = x - delta, x + delta
        gr[i] = (func(xf) - func(xb))/(2*h)
     
    return gr 
    
x = np.array([4.,3.]) 
print(f"FD:{central_difference(x, func=f1)}, AD:{grad_f(x)}")       

FD:[3.25      4.9899925], AD:[3.25      4.9899925]


## Autograd: Two Most Common Pitfalls 

#### Non-Float Arrays

Make sure you don't accidently try to autodiff using integers, or other objects!

In [9]:
x = np.array([4,3.]) # an integer array 
grad_f(x)


array([3.25     , 4.9899925])

#### Array Assignment
The following is a perfectly fine function...

In [10]:
def f2(x):
    n = A.shape[0]
    
    vals = np.zeros(n) 
    for i in range(n):
        vals[i] =  A[i,:] @ x.reshape(-1,1) # ***
        
    return np.sum(vals)

A = np.array([[1,2], [3,4]])
x = np.array([1.,2.])

# f-strings!
print(f"f2(x): {f2(x) :.2f}")


f2(x): 16.00


However, autograd cannot deal with *array assignment* (line with # ***). It will create a gradient function object just fine...

In [12]:
gradf2 = autograd.grad(f2,0) # creates a function object

but then when we go to do reverse mode it will throw this will throw a horrible error...

In [13]:
gradf2(x)

ValueError: setting an array element with a sequence.

When autograd says you are trying to set an *array element with a sequence*, it generally means you are trying to assign into an array - when it tries to build the graph it can't do it. 

**Solution:** *You should always try to vectorize first.* However, if you can't, or don't want to waste time, the solution is to use **list comprehensions**. List comprehensions (or any comprehensions) work just similar to how you would write set notation in maths. For example...

In [36]:
def f3(x): # f2 with list comprehension
    vals =  [A[i,:] @ x.reshape(-1,1) for i in range(A.shape[0])]    
    return np.sum(vals)

A = np.random.randn(100,2)
x = np.array([1.,2.])

# f-strings!
print(f"f(x): {f3(x) :.2f}")

gradf3 = autograd.grad(f3,0) # creates a function object
print(f"FD:{central_difference(x, func=f3)}, AD:{gradf3(x)}")  


f(x): -12.34
FD:[ 0.39866056 -6.37000628], AD:[ 0.39866056 -6.37000628]


Recall that for reverse mode we need to evaluate the function first to build the graph, so it would make sense to get the function and its gradient evaluated at $\mathbf{x}$

In [37]:
f3aux = autograd.value_and_grad(f3, 0)
val, grad = f3aux(x)

print('f:', val, ', gradf:', grad)

f: -12.341352006387245 , gradf: [ 0.39866056 -6.37000628]


If what we know about autodiff is correct, then ${\sf gradf3}$ and ${\sf f3aux}$ should be as fast as each other... 

In [38]:
%timeit -n 5 -r 25 gradf3(x)
%timeit -n 5 -r 25 f3aux(x)

15.2 ms ± 1.44 ms per loop (mean ± std. dev. of 25 runs, 5 loops each)
14.9 ms ± 1.06 ms per loop (mean ± std. dev. of 25 runs, 5 loops each)


How embarassing! :D 

## Something a bit more complicated!
Let's consider the function

$$f(A, c,x,y) = x^\top \cdot A\cdot x+\cdot \sin(y)^\top \cdot x,$$

where $A$ is a matrix,  $c$ is a scalar,  $x$ is a vector, $y$ is a vector.

In [41]:

%reset -f
import autograd 
import autograd.numpy as np 


def f(A, x, y):
    return x.T @ A @ x + np.sin(y).T @ x

A = np.array([[1.,2.],[2.,1.]])
x = np.array([[1.,2.]]).T
y = np.array([[1.,1.]]).T

val = f(A,x,y)
print(val)

g_A = autograd.grad(f,0)
g_x = autograd.grad(f,1)
g_y = autograd.grad(f,2)

print('gA:', g_A(A,x,y))
print('gx:', g_x(A,x,y))
print('gy:', g_y(A,x,y))

[[15.52441295]]
gA: [[1. 2.]
 [2. 4.]]
gx: [[10.84147098]
 [ 8.84147098]]
gy: [[0.54030231]
 [1.08060461]]


Unfortunately this requires 3x forward and backward passes through the graph. 

In [11]:
def vec(v):  return np.reshape(v,(-1,1),order="F")
def vecinv(v, shape): return np.reshape(v, shape, order="F")
def pack(collection): return np.vstack([vec(item) for item in collection])

shapes = {'A': A.shape, 'x' : x.shape, 'y' : y.shape}    
print(shapes) 

pack([A,x,y]) 

{'A': (2, 2), 'x': (2, 1), 'y': (2, 1)}


array([[1.],
       [2.],
       [2.],
       [1.],
       [1.],
       [2.],
       [1.],
       [1.]])

However, now we a new function that takes in the above vector as an argument, and "unpacks it" inside. 

## Hessians and Hessian-Vector Products
${\sf Autograd}$ his nice built in support for evaluating full Hessians, and Hessian-Vector Products...


In [45]:
def f3(x): # f2 with list comprehension
    vals =  [A[i,:] @ np.sin(x.reshape(-1,1)) for i in range(A.shape[0])]    
    return np.sum(vals)

d = 50000
A = np.random.rand(10,d)


np.random.seed(123)
x = 2*np.ones(d)
r = np.random.rand(d)

# create a function 
hvp_naive = lambda x,r : autograd.hessian(f3)(x) @ r

# autograd.hessian_vector_product returns a *function* with input (x,r)
hvp = autograd.hessian_vector_product(f3,argnum=0)

The difference is performance is highly non-trivial:

In [46]:
print('HVP...')
%timeit -n 1 -r 1 hvp(x, r)# hvp(x,r)
#print('\nForm Full Hessian and Multiply..')
#%timeit -n 10 -r 10 hvp_naive(x,r)

HVP...
25.5 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


-------
# PyTorch
**Advantages:** Faster than ${\sf autograd}$, GPU support, less quirks, probabilistic programming libraries, other modules built on it, lower level control. 

**Disadvantages:** Reverse mode only, with all documentation in that context, so need to know more autodiff theory to "get" it ( :D ), lower level control makes things a bit more involved.  

## Basics
${\sf Autograd}$ is ${\sf numpy}$-native. However, Pytorch has its own ${\sf Tensor}$ objects, which are used instead of ${\sf numpy}$ ${\sf ndarrays}$.

In [47]:
%reset -f 
import numpy as np 
import torch


x = torch.tensor([[1.],[2.],[3.]]) # column vector
x

tensor([[1.],
        [2.],
        [3.]])

Recently, support for the matrix multiplication operator @ and transpose attribute have been built in.

In [49]:
(x @ x.T).numpy()

array([[1., 2., 3.],
       [2., 4., 6.],
       [3., 6., 9.]], dtype=float32)

Which is very nice, because until more recent versions the above code would look like this:

In [16]:
torch.matmul(x, torch.transpose(x,0,1)) # the old way 

tensor([[1., 2., 3.],
        [2., 4., 6.],
        [3., 6., 9.]])

Any ${\sf Pytorch}$ ${\sf Tensor}$ object actually has a ${\sf .numpy}$ function, which casts it as a ${\sf numpy}$ array. 

In [17]:
x.numpy()

array([[1.],
       [2.],
       [3.]], dtype=float32)

Hang on, **float32**, whats with that?

In [50]:
x = torch.tensor([[1.],[2.],[3.]], dtype = torch.float64) # column vector
print(x.numpy(), x.numpy().dtype)


[[1.]
 [2.]
 [3.]] float64


Ok, now we are back to double precision!

**Generally**, ${\sf PyTorch}$ as the same syntax and functions as ${\sf Numpy}$, with occasionally minor differences: 


In [19]:
A = np.matmul(x.numpy(), np.transpose(x.numpy()))
B = torch.matmul(x, torch.transpose(x, 0, 1)) # PyTorch insists on you telling it which dimensions to swap 

print(B)
print(A) # numpy ndarrays 

tensor([[1., 2., 3.],
        [2., 4., 6.],
        [3., 6., 9.]], dtype=torch.float64)
[[1. 2. 3.]
 [2. 4. 6.]
 [3. 6. 9.]]


Pytorch builds the computation graph in the background by "watching" what you do to the tensors. However, you need to tell it to do the watching. This is done with the **requires_gradient** keyword argument. 

In [55]:
x = torch.tensor([[1.],[2.],[3.]],
                 dtype = torch.float64,
                 requires_grad=True)

x

tensor([[1.],
        [2.],
        [3.]], dtype=torch.float64, requires_grad=True)

Pytorch will now construct the graph in the background by logging the primal trace of any operations involving the tensor ${\sf x}$ during evaluation (forward pass).

In [56]:
y = x.T @ x 

In [57]:
torch.autograd.grad(y,x)
#torch.autograd.grad(y,x, grad_outputs = torch.tensor([[1.]]))

(tensor([[2.],
         [4.],
         [6.]], dtype=torch.float64),)

If we want to be nifty we can create our own "gradient_only" function...

In [59]:
f = lambda x: x.T @ x 
gr_f = lambda x: torch.autograd.grad(f(x),x) 


x = torch.tensor([[2],[2],[3.]], requires_grad=True)

f(x)
#gr_f(torch.tensor([[1],[2],[3]]) # spot the error!

tensor([[17.]], grad_fn=<MmBackward>)

So ${\sf grad}$ is the main function, but it assumes you have created the tensors and executed for forward pass first. 

### Gradients the "advanced" way: ${\sf .backward()}$

In [60]:
y = f(x)
y.backward()

In [61]:
x.grad

tensor([[4.],
        [4.],
        [6.]])

Be careful with this though, the function **accumulates** the adjoints in the ${\sf .grad}$ attribute of a tensor.

## Assignment is Not a Problem

## "Complicated" Example

$$f(A, c,x,y) = x^\top \cdot A\cdot x+\cdot \sin(y)^\top \cdot x$$

$A$ is a symmetric matrix,  $c$ is a scalar,  $x$ is a vector, $y$ is a vector.

Unlike with Autograd, we can backward through the graph and get all the gradients in one go. The ${\sf backward}$ function backpropagates the VJPs and stores the accumulated adjoint in the leaf nodes of the graph. 


In [ ]:
def f(A, x, y):
    z = x.T @ A @ x + torch.sin(y).T @ x
    z.backward()
    return z

dtype = torch.float64

A = torch.tensor(np.array([[1.,2.],[2.,1.]]), requires_grad=True, dtype = dtype)
x = torch.tensor(np.array([[1.,2.]]).T, requires_grad=True, dtype = dtype)
y = torch.tensor(np.array([[1.,1.]]).T, requires_grad=True, dtype = dtype)

val = f(A,x,y)

# mad dictionary skills
gradients = {'A': A.grad, 'x': x.grad, 'y': y.grad}

for k in gradients.keys():
    print(k, ':\n', gradients[k].numpy())


## Higher Order Derivatives

This is my attempt to make a function that computes the whole function by first creating a function  $g(\mathbf{x}) := \nabla f(\mathbf{x}$. Remember that the Hessian is just $\mathrm{J}_{\nabla f} = \mathrm{J}_g$. 

In [ ]:
import torch 

def f(x):
    return (x[0]**2 + x[0]*x[1])

def Hessian(f, x):
    g = torch.autograd.grad(f(x), x, create_graph=True)[0]
    d = x.shape[0]
    rows = [] 
    for i in range(d):
        vec = torch.zeros(d)
        vec[i] = 1

        g.backward(vec, retain_graph = True) # vector-Jacobian products! (Hessian Rows)
        rows.append(x.grad.clone())
        x.grad.zero_() # zeroing out the gradients in the leafs
        H = torch.stack(rows)
    
    return H 

x = torch.tensor([1.,2.], requires_grad=True)

Hessian(f, x)

#### Performance
${\sf PyTorch}$ has a tendency to be faster than ${\sf Numpy}$ even on a CPU, particularly for large matrices. Compare creating ${\rm A}$ and ${\rm B}$ as $d \times d$ standard normal matrices, and computing their outer product ${\rm A}{\rm B}^\top$. I do it the obvious way, but also using a special function called ${\sf einsum}$. 

In [62]:
%reset -f
import numpy as np
import torch 

d = 500

print("Numpy...")
%timeit -n 5 -r 5 np.einsum('ik, jk ->j', np.random.randn(d,d), np.random.randn(d,d))
%timeit -n 5 -r 5 np.random.randn(d,d) @ np.random.randn(d,d).T

print("Pytorch...")
%timeit -n 5 -r 5 torch.einsum('ik, jk -> j', torch.randn(d,d), torch.randn(d,d))
%timeit -n 5 -r 5 torch.randn(d,d) @ torch.randn(d,d). T



Numpy...
81.1 ms ± 2.74 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)
16.3 ms ± 793 µs per loop (mean ± std. dev. of 5 runs, 5 loops each)
Pytorch...
3.26 ms ± 1.15 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)
3.69 ms ± 458 µs per loop (mean ± std. dev. of 5 runs, 5 loops each)


## Demonstration: Calling Autodiff'd Python Functions in ${\sf R}$

One can call Python functions in {\sf R} using the ${\sf reticulate}$ package. I will do a quick demo using my ${\sf{reticdemo.r}}$ and ${\sf func.py}$ files.

My advice for ${\sf Python}$/${\sf R}$ interoperability is as follows: 

(1) Reticulate will convert R objects and will send Python a ${\sf numpy}$ array --- make sure your function(s) transform them into tensors on input (use ${\sf torch.tensor}()$)

(2) Make sure the function returns a numpy array for each output --- use ${\sf .numpy()}$ or ${\sf .data.numpy()}$ on the PyTorch tensors before returning them.



## Exercises
*(1)* Implement ${\sf f2}$ from the autograd example in Pytorch, but use array assignment,call this function  ${\sf f2a}$, then do it with a list comprehension, calling it  ${\sf f2b}$. Compare the speed of the two. 


*(2)* Using PyTorch/Autograd, create a function that *explicitly* forms the full **Jacobian** for the function $f(x_1, x_2) = (x_1, x_1 x_2)$. You will need an array stacking function. Pass in some numeric values and ensure the answer matches.

*(3)* Fully implement the ''complicated'' example in Autograd in a way that only requires one function evaluation to get all gradients, and returns a vector of these gradients. You will need to pack elements going in, and unpack the outputs full gradient vector it outputs - i.e., you have the same output as if you did three forward and backward passes of autodiff. (You will need to make an ${\sf unpack}$ function. 

(4) Investigate what the ${\sf einsum}$ function does (same in ${\sf numpy}$ and ${\sf PyTorch}$) - use it to get (1) row sum of a random matrix , (2) column sum of a random matrix, and (3) matrix product of two random matrices. Compare with the direct implementation using standard linear algebra functions. 

(5) Explore ${\sf reticulate}$ further and use it to call ${\sf f1}$ and its gradient from ${\sf R}$. 

(6) **Challenge Exercise**: Defining New Primatives in Autograd.

${\sf Autograd}$ does not have build in support to autodiff the **probit** function $\Phi^{-1}(x), x \in (0,1)$, so lets implement it! It is simple to show that 
$$\frac{{\rm d}}{{\rm d}x}\Phi^{-1}(x) = \frac{1}{\phi\big(\Phi^{-1}(x)\big)}.$$

Consider the function evaluated elementwise for an input: $\mathbf{\Phi}^{-1}(x), \mathbf{x} \in (0,1)^n$, so $\mathbf{\Phi}^{-1}:(0,1)^n \rightarrow \mathbb{R}^n$. 

Then, 
 $$\mathrm{J}_{\mathbf{\Phi}^{-1}}(\mathbf{x}) = {\rm Diag\left(\frac{1}{\mathbf{\phi}\big(\mathbf{\Phi}^{-1}(\mathbf{x})\big)} \right)},$$

 and thus 
 
 $${\sf vjp}_{\mathbf{\Phi^{-1}}}(\mathbf{x},\mathbf{r}) = \mathbf{r}^\top\mathrm{J}_{\Phi^{-1}}(\mathbf{x}) = [\mathrm{J}_{\Phi^{-1}}(\mathbf{x})^\top \mathbf{r}]^\top = \left[\frac{\mathbf{r}}{\mathbf{\phi}\big(\mathbf{\Phi}^{-1}(\mathbf{x})\big)}\right]^\top.$$
 
Visit the ${\sf autograd}$ Github page, learn how to create new primitives using the ${\sf @primitive}$ decorator and ${\sf defvjp}$ function, and implement these for a new primitive called ${\sf probit}$. Defining new primitives is similar in ${\sf PyTorch}$, albeit requiring more Python skills. 

## Extra

In [16]:
# Numpy Example: Pass by Reference

import numpy as np

def f(y):
  y = y**2 

x = np.array([5])
f(x)

print(x) 



[5]


In [30]:
# Numpy Example: Broadcasting

x = np.array([1,2])
y = np.array([[1,2]]).T

x = x.reshape(-1,1)

In [31]:
x

array([[1],
       [2]])

In [32]:
y

array([[1],
       [2]])

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 2 is different from 1)